[![ebac_logo-data_science.png](https://raw.githubusercontent.com/rhatiro/Curso_EBAC-Profissao_Cientista_de_Dados/main/ebac-course-utils/media/logo/ebac_logo-data_science.png)](https://github.com/rhatiro/Curso_EBAC-Profissao_Cientista_de_Dados)
<!-- <img src="https://raw.githubusercontent.com/rhatiro/Curso_EBAC-Profissao_Cientista_de_Dados/main/ebac-course-utils/media/logo/ebac_logo-data_science.png" alt="ebac_logo-data_science"> -->

---

<!-- # **Profissão: Cientista de Dados** -->
### **Módulo 23** | Combinação de modelos I | Exercício 1

**Aluno:** [Roberto Hatiro Nishiyama](https://www.linkedin.com/in/rhatiro/)<br>
**Data:** 30 de maio de 2023.

---

# Tarefa 01

**1.** Monte um passo a passo para o Bagging

> O ***Bagging (Bootstrap Aggregating)*** consiste nos seguintes passos:
> 1. ***Bootstrap:*** O processo de bootstrap envolve a criação de amostras aleatórias de treinamento com reposição a partir do conjunto de dados de treinamento original. Isso significa que cada amostra bootstrap tem o mesmo tamanho do conjunto de dados original, mas é obtida permitindo que cada exemplo seja selecionado várias vezes.
> 2. **Modelagem (*Base learners*):** Para cada amostra bootstrap criada, um modelo de *Machine Learning* é treinado de forma independente. Isso significa que múltiplos modelos são criados, onde cada um é treinado em uma amostra bootstrap diferente. Esses modelos são chamados de *base learners*.
> 3. **Agregação (*Aggregating*):** Nesta etapa, os resultados dos modelos individuais são agregados para formar uma única previsão final. No caso de problemas de classificação, a agregação geralmente é feita por votação majoritária, onde a classe mais comum entre os modelos é selecionada como a previsão final. Para problemas de regressão, a agregação é feita calculando-se a média dos resultados dos modelos.

**2.** Explique com suas palavras o Bagging

> O ***Bagging***, ou ***Bootstrap Aggregating***, é um método de combinação de modelos usado em *Machine Learning*. Ele envolve a criação de várias amostras de treinamento, onde cada amostra é gerada aleatoriamente com reposição a partir do conjunto de dados original, em um processo chamado *Bootstrap*. Em seguida, cada amostra é usada para treinar um modelo independente, chamado de *base learner*.
>
> Na etapa de teste, os modelos são aplicados aos dados de teste e suas previsões são combinadas por meio de uma estratégia de agregação. No caso de problemas de classificação, a combinação geralmente é feita por votação majoritária, ou seja, a classe mais frequente entre os modelos é escolhida como a classe final. No caso de problemas de regressão, a combinação é feita calculando-se a média das previsões dos modelos.

**3.** (Opcional) Implementar em python o código do Bagging
   - Bootstrap
   - Modelagem
   - Agregação

In [1]:
# Import das bibliotecas

import pandas as pd

from sklearn.datasets        import load_iris
from sklearn.datasets        import load_diabetes

from sklearn.model_selection import train_test_split

from sklearn.tree            import DecisionTreeClassifier
from sklearn.metrics         import accuracy_score

from sklearn.tree            import DecisionTreeRegressor
from sklearn.metrics         import mean_squared_error, r2_score

In [2]:
# Exemplo da técnica de bagging para problemas de classificação

X = load_iris().data
y = load_iris().target

df = pd.DataFrame(X, columns=load_iris().feature_names)
df['target'] = y

def bagging_classifier(df:pd.DataFrame, 
                       num_bootstrap_samples:int=3,  # Parâmetro da função que define a quantidade de amostragens para treinamento
                       test_size:float=0.25
                      ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # Dicionário para os resultados das predições de cada modelo
    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)  # Amostragem COM reposição

        X_train = df_train.drop(['target'], axis=1)
        y_train = df_train['target']
        
        # Modelagem (base learners)
        model = DecisionTreeClassifier()
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao dicionário para agregação das predições
        y_pred_bagging.update({i:model.predict(X_test)})
    
    # Aggregating
    y_pred = (pd.DataFrame(y_pred_bagging)
                .mode(axis=1)  # Agregando o valor com maior número de aparições nas predições dos modelos
                .rename(columns={0:'y_pred'}))
 
    # Resultados
    print(model)
    print('Accuracy score:', accuracy_score(y_true=y_test, 
                                            y_pred=y_pred['y_pred']
                                           ))

    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred['y_pred'].astype(int)], 
                     axis=1)

bagging_classifier(num_bootstrap_samples=10, df=df, test_size=0.33)

DecisionTreeClassifier()
Accuracy score: 0.92


,y_test,y_pred
0,2,2
1,1,1
2,0,0
3,2,1
4,1,1
5,1,1
6,0,0
7,2,1
8,0,0
9,0,0


In [3]:
# Exemplo da técnica de bagging para problemas de regressão

X = load_diabetes().data
y = load_diabetes().target

df = pd.DataFrame(X, columns=load_diabetes().feature_names)
df['target'] = y

def bagging_regressor(df:pd.DataFrame, 
                      num_bootstrap_samples:int=3,  # Parâmetro da função que define a quantidade de amostragens para treinamento
                      test_size:float=0.25
                      ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # Dicionário para os resultados das predições de cada modelo
    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)  # Amostragem COM reposição

        X_train = df_train.drop(['target'], axis=1)
        y_train = df_train['target']

        # Modelagem (base learners)
        model = DecisionTreeRegressor()
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao dicionário para agregação das predições
        y_pred_bagging.update({i:model.predict(X_test)})

    # Aggregating
    y_pred = (pd.DataFrame(y_pred_bagging)
                .mean(axis=1)  # Agregando as predições dos modelos baseando n a média dos resultados
                .rename('y_pred'))
 
    # Resultados
    print(model)
    print('Mean squared error:', mean_squared_error(y_true=y_test, 
                                                   y_pred=y_pred))
    print('Coefficient of determination:', r2_score(y_true=y_test, 
                                                    y_pred=y_pred))
    
    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred], 
                     axis=1)
    
bagging_regressor(num_bootstrap_samples=100, df=df, test_size=0.33)

DecisionTreeRegressor()
Mean squared error: 3495.8629835616434
Coefficient of determination: 0.3819796763415946


,y_test,y_pred
0,246.0,186.49
1,248.0,203.83
2,182.0,117.78
3,142.0,184.31
4,154.0,136.21
...,...,...
141,172.0,180.12
142,185.0,137.12
143,67.0,159.76
144,97.0,98.48


---